# Convert trajectories to .csv files

In [1]:
import numpy as np
import os
import pickle

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [4]:
from tqdm.auto import tqdm
from os import mkdir
from os.path import exists, join, split
import yaml

Select folder where decay trajectories (.pkl files) are located and location of rest file (containing equilibrium position)

In [5]:
# 000, 001, 002, ...
locs = [f"{i:03d}" for i in range(100)]
decayData_dir = [f"/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_10ms_N=100/{loc}/decay" for loc in locs]

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = os.path.join(robot_dir, 'rest_qv.pkl')

output_node = 'all' # 1354 # 51 #
TIP_NODE = 51
t_in = 1
t_out = 4

Save data to .csv (one file per trajectory)

In [12]:
start_at = 0
end_at = 1
for i, dir in enumerate(tqdm(decayData_dir[start_at:end_at])):
    loc = locs[start_at:end_at][i]
    roshan_dir = f"{dir}/roshan_{loc}"
    if not exists(roshan_dir):
        mkdir(roshan_dir)
    oData = utils.import_pos_data(dir, rest_file=rest_file, output_node=output_node, t_in=t_in, t_out=t_out, return_velocity=True)
    for j in tqdm(range(len(oData)), leave=False):
        if j == 0:
            # save time array only once, as it is the same for all trajectories
            np.savetxt(f"{roshan_dir}/t_{loc}.csv", oData[j][0], fmt='%.5e')
        np.savetxt(f"{roshan_dir}/decay_trajectory_{loc}_{j:02d}.csv", oData[j][1], fmt='%.5e')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Add rest position and pre-tensioning input for each SSM location

In [8]:
for i, dir in enumerate(tqdm(decayData_dir)):
    loc = locs[i]
    roshan_dir = f"{dir}/roshan_{loc}"
    if not exists(roshan_dir):
        mkdir(roshan_dir)
    with open(join(dir, "..", "rest_q.pkl"), "rb") as f:
        rest_q = pickle.load(f)
    with open(join(dir, "..", "pre_tensioning.pkl"), "rb") as f:
        pre_tensioning = pickle.load(f)
    # save rest_q array to csv
    np.savetxt(f"{roshan_dir}/rest_pos_{loc}.csv", rest_q, fmt='%.5e')
    np.savetxt(f"{roshan_dir}/pre_tensioning_u_{loc}.csv", pre_tensioning, fmt='%.5e')

  0%|          | 0/100 [00:00<?, ?it/s]

Compress all the data

In [13]:
import tarfile
import os.path

out_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/data_roshan_trunk_adiabatic_10ms_N=100"
if not exists(out_dir):
    mkdir(out_dir)
start_at = 0
end_at = 1
for i, dir in enumerate(tqdm(decayData_dir[start_at:end_at])):
    loc = locs[start_at:end_at][i]
    roshan_dir = f"{dir}/roshan_{loc}"
    with tarfile.open(join(out_dir, f"{loc}.tar.gz"), "w:gz") as tar:
        tar.add(roshan_dir, arcname=os.path.basename(roshan_dir))

  0%|          | 0/1 [00:00<?, ?it/s]

Export open-loop circle trajectories to .csv for Roshan

In [ ]:
traj_dirs = [
    "/media/jonas/Backup Plus/jonas_soft_robot_data/autonomous_ASSM_tests/OL_sim_ideal",
    "/media/jonas/Backup Plus/jonas_soft_robot_data/autonomous_ASSM_tests/OL_sim_perturbed"
]
alphas = [0.1, 0.2, 0.4, 0.7, 1.0, 1.5, 2.0, 3.0]
output_node = 'all' # TIP_NODE
for i, dir in enumerate(tqdm(traj_dirs)):
    for alpha in tqdm(alphas):
        dir_alpha = join(dir, f"alpha={alpha:.1f}")
        roshan_dir = join(dir_alpha, "roshan")
        if not exists(roshan_dir):
            mkdir(roshan_dir)
        oData = utils.import_pos_data(dir_alpha, rest_file, output_node=output_node, t_in=3, return_velocity=True)
        np.savetxt(f"{roshan_dir}/t.csv", oData[0], fmt='%.5e')
        np.savetxt(f"{roshan_dir}/{split(dir)[1]}.csv", oData[1], fmt='%.5e')